In [1]:
import cv2
import pandas as pd
import numpy as np

In [ ]:
def preProcessing0(dataset):
    output = []
        for j in range(len(dataset)):
        temp = dataset[j]
        temp[temp < 251] = 0
        temp[temp >= 251] = 255
        output.append(temp.ravel())
    return output

# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

# proc_train_x = pd.DataFrame(preProcessing0(train_x))
# proc_test_x = pd.DataFrame(preProcessing0(test_x))

# proc_train_x.to_csv('train_x_processed0.csv', header=None, index=None)
# proc_test_x.to_csv('test_x_processed0.csv', header=None, index=None)

In [ ]:
def preProcessing1(dataset):
    output = []
    for j in range(len(dataset)):
        thresh = cv2.threshold(dataset[j], 251, 255, cv2.THRESH_BINARY)[1]
        img, cnts, heir = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        area_list = []

        for c in cnts:
            (x_i, y_i, w, h) = cv2.boundingRect(c)
            temp_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
            squareDim = np.max(temp_roi.shape)
            area_list.append(squareDim**2)

        idx = area_list.index(max(area_list)) 

        (x_i, y_i, w, h) = cv2.boundingRect(cnts[idx])
        new_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
        
        temp = dataset[j] 
        temp[temp > -1] = 0  
        temp[((64-new_roi.shape[0])//2):(((64-new_roi.shape[0])//2)+new_roi.shape[0]), ((64-new_roi.shape[1])//2):(((64-new_roi.shape[1])//2)+new_roi.shape[1])] = new_roi
        output.append(temp.ravel())
    return output

# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

# proc_train_x = pd.DataFrame(preProcessing1(train_x))
# proc_test_x = pd.DataFrame(preProcessing1(test_x))

# proc_train_x.to_csv('train_x_processed1.csv', header=None, index=None)
# proc_test_x.to_csv('test_x_processed1.csv', header=None, index=None)

In [ ]:
def preProcessing1b(dataset):
    output = []
    for j in range(len(dataset)):
        thresh = cv2.threshold(dataset[j], 251, 255, cv2.THRESH_BINARY)[1]
        img, cnts, heir = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        area_list = []

        for c in cnts:
            (x_i, y_i, w, h) = cv2.boundingRect(c)
            temp_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
            squareDim = np.max(temp_roi.shape)
            area_list.append(squareDim**2)

        idx = area_list.index(max(area_list)) 

        ii = np.where(area_list == max(area_list))[0]
        if len(ii) > 1:
            smallestContourArea = cv2.boundingRect(cnts[ii[0]])
            sca_idx = ii[0]
            for i in range(1,len(ii)):
                if smallestContourArea > cv2.boundingRect(cnts[ii[i]]):
                    smallestContourArea = cv2.boundingRect(cnts[ii[i]])
                    sca_idx = ii[i]
            idx = sca_idx

        (x_i, y_i, w, h) = cv2.boundingRect(cnts[idx])
        new_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]

        temp = dataset[j] 
        temp[temp > -1] = 0  
        temp[((64-new_roi.shape[0])//2):(((64-new_roi.shape[0])//2)+new_roi.shape[0]), ((64-new_roi.shape[1])//2):(((64-new_roi.shape[1])//2)+new_roi.shape[1])] = new_roi
        output.append(temp.ravel())
    return output

# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

# proc_train_x = pd.DataFrame(preProcessing1b(train_x))
# proc_test_x = pd.DataFrame(preProcessing1b(test_x))

# proc_train_x.to_csv('train_x_processed1b.csv', header=None, index=None)
# proc_test_x.to_csv('test_x_processed1b.csv', header=None, index=None)

In [ ]:
def deskew(img):
    SZ=64
    affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

def preProcessing2a(dataset):
    output = []
    for j in range(len(dataset)):
        thresh = cv2.threshold(dataset[j], 251, 255, cv2.THRESH_BINARY)[1]

        img, cnts, heir = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        area_list = []

        for c in cnts:
            (x_i, y_i, w, h) = cv2.boundingRect(c)
            temp_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
            squareDim = np.max(temp_roi.shape)
            area_list.append(squareDim**2)

        idx = area_list.index(max(area_list)) 

        ii = np.where(area_list == max(area_list))[0]
        if len(ii) > 1:
            smallestContourArea = cv2.boundingRect(cnts[ii[0]])
            sca_idx = ii[0]
            for i in range(1,len(ii)):
                if smallestContourArea > cv2.boundingRect(cnts[ii[i]]):
                    smallestContourArea = cv2.boundingRect(cnts[ii[i]])
                    sca_idx = ii[i]
            idx = sca_idx

        (x_i, y_i, w, h) = cv2.boundingRect(cnts[idx])
        new_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]

        temp = dataset[j] 
        temp[temp > -1] = 0  
        temp[((64-new_roi.shape[0])//2):(((64-new_roi.shape[0])//2)+new_roi.shape[0]), ((64-new_roi.shape[1])//2):(((64-new_roi.shape[1])//2)+new_roi.shape[1])] = new_roi
        
        coords = np.column_stack(np.where(temp > 0))
        angle = cv2.minAreaRect(coords)[-1]
        
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle

        # rotate the image to deskew it
        (h, w) = temp.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(temp, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        final = deskew(rotated)
        output.append(final.ravel())
    return output

# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

# proc_train_x = pd.DataFrame(preProcessing2a(train_x))
# proc_test_x = pd.DataFrame(preProcessing2a(test_x))

# proc_train_x.to_csv('train_x_processed2a.csv', header=None, index=None)
# proc_test_x.to_csv('test_x_processed2a.csv', header=None, index=None)

In [ ]:
def deskew(img):
    SZ=64
    affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

def preProcessing2b(dataset):
    output = []
    for j in range(len(dataset)):
        thresh = cv2.threshold(dataset[j], 251, 255, cv2.THRESH_BINARY)[1]

        img, cnts, heir = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        area_list = []

        for c in cnts:
            (x_i, y_i, w, h) = cv2.boundingRect(c)
            temp_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
            squareDim = np.max(temp_roi.shape)
            area_list.append(squareDim**2)

        idx = area_list.index(max(area_list)) 

        ii = np.where(area_list == max(area_list))[0]
        if len(ii) > 1:
            smallestContourArea = cv2.boundingRect(cnts[ii[0]])
            sca_idx = ii[0]
            for i in range(1,len(ii)):
                if smallestContourArea > cv2.boundingRect(cnts[ii[i]]):
                    smallestContourArea = cv2.boundingRect(cnts[ii[i]])
                    sca_idx = ii[i]
            idx = sca_idx

        (x_i, y_i, w, h) = cv2.boundingRect(cnts[idx])
        new_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]

        temp = dataset[j] 
        temp[temp > -1] = 0  
        temp[((64-new_roi.shape[0])//2):(((64-new_roi.shape[0])//2)+new_roi.shape[0]), ((64-new_roi.shape[1])//2):(((64-new_roi.shape[1])//2)+new_roi.shape[1])] = new_roi
        
        angles = np.linspace(-45, 45, num=91).astype(int)
        rotate_area_list = []
    
        for i in range(len(angles)):
            rotateTest = temp
            (h, w) = rotateTest.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, angles[i], 1.0)
            rotated = cv2.warpAffine(temp, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
            img, cnts, heir = cv2.findContours(rotated.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnt_w = []
            for c in cnts:
                (x_i, y_i, w, h) = cv2.boundingRect(c)
                cnt_w.append(w)
            rotate_area_list.append(max(cnt_w))

        rotate_idx = rotate_area_list.index(min(rotate_area_list))

        (h, w) = temp.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angles[rotate_idx], 1.0)
        rotated = cv2.warpAffine(temp, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        final = deskew(rotated)
        output.append(final.ravel())
        print(j)
    return output

# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

# proc_train_x = pd.DataFrame(preProcessing2b(train_x))
# proc_test_x = pd.DataFrame(preProcessing2b(test_x))

# proc_train_x.to_csv('train_x_processed2b.csv', header=None, index=None)
# proc_test_x.to_csv('test_x_processed2b.csv', header=None, index=None)

In [ ]:
import csv
import requests
from io import StringIO

def deskew(img):
    SZ=64
    affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

def preProcessing2c(dataset):
    output = []
    for j in range(len(dataset)):
        thresh = cv2.threshold(dataset[j], 251, 255, cv2.THRESH_BINARY)[1]

        img, cnts, heir = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        area_list = []
        #temp2 = dataset[j]
        temp2 = np.zeros((64,64))
        bestAng = []
        for c in cnts:
            (x_i, y_i, w, h) = cv2.boundingRect(c)
            temp_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
            temp2[temp2 > -1] = 0 
            temp2[((64-temp_roi.shape[0])//2):(((64-temp_roi.shape[0])//2)+temp_roi.shape[0]), ((64-temp_roi.shape[1])//2):(((64-temp_roi.shape[1])//2)+temp_roi.shape[1])] = temp_roi
            angles = np.linspace(-45, 45, num=91).astype(int)
            rotate_w, rotate_h = [], []
            for i in range(len(angles)):
                rotateTest = temp2
                (h, w) = rotateTest.shape[:2]
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, angles[i], 1.0)
                rotated = cv2.warpAffine(rotateTest, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
                img2, cnts2, heir2 = cv2.findContours(rotated.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                cnt_w = []
                cnt_h = []
                for c1 in cnts2:
                    (x_i, y_i, w, h) = cv2.boundingRect(c1)
                    cnt_w.append(w)
                    cnt_h.append(h)
                rotate_w.append(max(cnt_w))
                rotate_h.append(cnt_h[cnt_w.index(max(cnt_w))])
                
            rotate_idx = rotate_w.index(min(rotate_w))
            minW = rotate_w[rotate_idx]
            hFromMinW = rotate_h[rotate_idx]
            squareDim = max(minW, hFromMinW)
            area_list.append(squareDim**2)
            bestAng.append(angles[rotate_idx])
        idx = area_list.index(max(area_list))
        ii = np.where(area_list == max(area_list))[0]
        
        (x_i, y_i, w, h) = cv2.boundingRect(cnts[idx])
        new_roi = thresh[y_i:(y_i+h), x_i:(x_i+w)]
        temp = np.zeros((64,64))
        temp[temp > -1] = 0  
        temp[((64-new_roi.shape[0])//2):(((64-new_roi.shape[0])//2)+new_roi.shape[0]), ((64-new_roi.shape[1])//2):(((64-new_roi.shape[1])//2)+new_roi.shape[1])] = new_roi
        angles = np.linspace(-45, 45, num=91).astype(int)
        rotate_area_list = []
    
        (h, w) = temp.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, bestAng[idx], 1.0)
        rotated = cv2.warpAffine(temp, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        final = deskew(rotated)
        output.append(final.ravel())
        print(j)
    return output

# print("Start loading data ...")
# train_x = np.loadtxt("trainset/train_x.csv", delimiter=",") # load from text 
# train_y = np.loadtxt("trainset/train_y.csv", delimiter=",")
# test_x = np.loadtxt("trainset/test_x.csv", delimiter=",")
# train_x = train_x.reshape(-1, 64, 64) # reshape 
# train_y = train_y.reshape(-1, 1)
# test_x = test_x.reshape(-1, 64, 64) # reshape 

train_x_url = 'http://cs.mcgill.ca/~ksinha4/datasets/kaggle/train_x.csv'
train_y_url = 'http://cs.mcgill.ca/~ksinha4/datasets/kaggle/train_y.csv'
test_x_url = 'http://cs.mcgill.ca/~ksinha4/datasets/kaggle/test_x.csv'

print("Downloading train_x.csv From URL . . .")
with requests.Session() as s:
    download = s.get(train_x_url)
    train_x = pd.read_csv(StringIO(download.text))

print("Downloading train_y.csv From URL . . .")
with requests.Session() as s:
    download = s.get(train_y_url)
    train_y = pd.read_csv(StringIO(download.text))

print("Downloading test_x.csv From URL . . .")
with requests.Session() as s:
    download = s.get(test_x_url)
    test_x = pd.read_csv(StringIO(download.text))

train_x = train_x.values.reshape(-1, 64, 64) # reshape 
train_y = train_y.reshape(-1, 1)
test_x = test_x.values.reshape(-1, 64, 64) # reshape     
    
print("Preprocessing train ... ")
proc_train_x = pd.DataFrame(preProcessing2c(train_x))
print("Preprocessing test ... ")
proc_test_x = pd.DataFrame(preProcessing2c(test_x))

proc_train_x.to_csv('train_x_processed2c.csv', header=None, index=None)
proc_test_x.to_csv('test_x_processed2c.csv', header=None, index=None)
test_x.to_csv('train_y.csv', header=None, index=None)

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler

#X_std = StandardScaler().fit_transform(train_x)
#cov_mat = np.cov(X_std.T)
cov_mat = np.cov(train_x.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

In [4]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,4095)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,4095)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
iplot(fig)

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
norm_train_x = scaler.fit_transform(train_x) #Normalize train_x
norm_test_x = scaler.transform(test_x) #Normalize test_x

X_std = StandardScaler().fit_transform(train_x)
pca = PCA(n_components=1024)
pca_train_x = pca.fit_transform(X_std)
pca_test_x = pca.transform(test_x)

# pca = PCA(n_components=1024)
# pca_train_x = pca.fit_transform(train_x)
# pca_test_x = pca.transform(test_x)